In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('TradeData_6_15_2025_16_41_2.csv', encoding='latin1')

In [1]:
import sqlite3

In [7]:
# Create/connect to SQLite DB (creates if not exists)
conn = sqlite3.connect("bursa_palmai_database.db")  

# Write DataFrame to SQLite (creates a table named 'mth_prod_data')
df.to_sql(
    name="test_gt",   # Table name
    con=conn,            # SQLite connection
    if_exists="replace", # Overwrite if table exists
    index=False          # Don't write DataFrame index
)

# Confirm
print("Table successfully created in SQLite!")

Table successfully created in SQLite!


In [9]:
conn = sqlite3.connect("bursa_palmai_database.db")  
query_result = pd.read_sql("SELECT * FROM test_gt limit 10", conn)
print(query_result)

  typeCode  freqCode  refPeriodId  refYear  refMonth  period reporterCode  \
0        A  20130101         2013       52      2013     188          CRI   
1        A  20130101         2013       52      2013     188          CRI   
2        A  20130101         2013       52      2013     188          CRI   
3        A  20130101         2013       52      2013     188          CRI   
4        A  20130101         2013       52      2013     188          CRI   
5        A  20130101         2013       52      2013     188          CRI   
6        A  20130101         2013       52      2013     188          CRI   
7        A  20130101         2013       52      2013     188          CRI   
8        A  20130101         2013       52      2013     188          CRI   
9        A  20130101         2013       52      2013     188          CRI   

  reporterISO reporterDesc flowCode  ...  netWgt isNetWgtEstimated grossWgt  \
0  Costa Rica            M   Import  ...       0              None       

In [33]:
import sqlite3

def list_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query SQLite's internal schema table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    conn.close()
    return [table[0] for table in tables]

# Usage
tables = list_tables("bursa_palmai_database.db")
print("Tables in database:", tables)

Tables in database: ['company_mthly_prod', 'company_ext_rate', 'company_plt_area', 'mpob_stats', 'universal_mill_list']


In [34]:
def get_table_schema(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    conn.close()
    return schema

# Usage
print(get_table_schema("bursa_palmai_database.db", "company_plt_area"))

[(0, 'Company', 'TEXT', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Year', 'INTEGER', 0, None, 0), (3, 'Value', 'INTEGER', 0, None, 0)]


#Concession weather forecast

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import geodesic
import requests

response = requests.get('https://api.data.gov.my/weather/forecast')
wfcast_json = response.json()
wfcast_df = pd.json_normalize(wfcast_json)

wfcast_df = wfcast_df[['date', 'summary_forecast', 'min_temp', 'max_temp', 'location.location_name']]
wfcast_df.rename(columns={'location.location_name': 'location_name'}, inplace=True)
points_df = pd.read_csv('weather_station_base.csv')

rain_table = wfcast_df.merge(points_df, on='location_name', how='left')
weather_gdf = gpd.GeoDataFrame(
    rain_table,
    geometry=gpd.points_from_xy(rain_table.Longitude, rain_table.Latitude),
    crs="EPSG:4326"
)
weather_gdf['date'] = pd.to_datetime(weather_gdf['date'])

earliest_date = weather_gdf['date'].min()

concessions = gpd.read_file("rspo_oil_palm/rspo_oil_palm_v20200114.shp").to_crs("EPSG:4326")
concessions = concessions[concessions['country'].isin(['Malaysia'])]

station_points = weather_gdf[['location_name', 'Longitude', 'Latitude']].drop_duplicates()
station_points['geometry'] = gpd.points_from_xy(station_points.Longitude, station_points.Latitude)
station_gdf = gpd.GeoDataFrame(station_points, geometry='geometry', crs='EPSG:4326')

# Find nearest station and calculate distance
def get_nearest_station_info(row):
    concession_centroid = row.geometry.centroid
    nearest_station = station_gdf.geometry.distance(concession_centroid).sort_values().index[0]
    nearest_row = station_gdf.loc[nearest_station]
    
    # Compute geodesic distance in km
    dist_km = geodesic(
        (concession_centroid.y, concession_centroid.x),
        (nearest_row.Latitude, nearest_row.Longitude)
    ).km
    
    return pd.Series({
        'nearest_station': nearest_row.location_name,
        'distance_km': round(dist_km, 2)
    })

# Apply to concessions
concessions[['nearest_station', 'distance_km']] = concessions.apply(get_nearest_station_info, axis=1)

# Merge 7-day forecast from the matched station
concessions_forecast = pd.merge(
    concessions,
    weather_gdf,
    left_on='nearest_station',
    right_on='location_name',
    how='left'
)

concessions_forecast_comp = concessions_forecast.reset_index(drop=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_20460\2450329422.py:36: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_station = station_gdf.geometry.distance(concession_centroid).sort_values().index[0]


In [6]:
concessions_forecast_comp

,gfw_fid,objectid,group_,company,plantation,rspocert,memberyear,state,province,country,shape_leng,shape_area,gfw_area__,gfw_geosto,created_on,updated_on,geometry_x,nearest_station,distance_km,date,summary_forecast,min_temp,max_temp,location_name,Latitude,Longitude,geometry_y
0,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.10,2025-06-21,Ribut petir di kebanyakan tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
1,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.10,2025-06-20,Hujan di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
2,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.10,2025-06-19,Hujan di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
3,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.10,2025-06-18,Hujan di kebanyakan tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
4,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.10,2025-06-17,Ribut petir di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5539,1846,1863,Wild Asia Group Scheme,Sudirman Bin Sapie,Sudirman Bin Sapie,True,None,Sabah,Kinabatangan,Malaysia,5.275587,0.002255,2762.596976,2ade5c88-cd16-c279-598f-4a9eeefb7c45,2020-07-28,2020-07-28,"MULTIPOLYGON (((101.30821 4.02156, 101.30816 4...",Kuala Penyu,10.37,2025-06-19,Ribut petir di kebanyakan tempat,24,33,Kuala Penyu,5.5717,115.5971,POINT (115.5971 5.5717)
5540,1846,1863,Wild Asia Group Scheme,Sudirman Bin Sapie,Sudirman Bin Sapie,True,None,Sabah,Kinabatangan,Malaysia,5.275587,0.002255,2762.596976,2ade5c88-cd16-c279-598f-4a9eeefb7c45,2020-07-28,2020-07-28,"MULTIPOLYGON (((101.30821 4.02156, 101.30816 4...",Kuala Penyu,10.37,2025-06-18,Ribut petir di kebanyakan tempat,24,33,Kuala Penyu,5.5717,115.5971,POINT (115.5971 5.5717)
5541,1846,1863,Wild Asia Group Scheme,Sudirman Bin Sapie,Sudirman Bin Sapie,True,None,Sabah,Kinabatangan,Malaysia,5.275587,0.002255,2762.596976,2ade5c88-cd16-c279-598f-4a9eeefb7c45,2020-07-28,2020-07-28,"MULTIPOLYGON (((101.30821 4.02156, 101.30816 4...",Kuala Penyu,10.37,2025-06-17,Ribut petir di kebanyakan tempat,24,33,Kuala Penyu,5.5717,115.5971,POINT (115.5971 5.5717)
5542,1846,1863,Wild Asia Group Scheme,Sudirman Bin Sapie,Sudirman Bin Sapie,True,None,Sabah,Kinabatangan,Malaysia,5.275587,0.002255,2762.596976,2ade5c88-cd16-c279-598f-4a9eeefb7c45,2020-07-28,2020-07-28,"MULTIPOLYGON (((101.30821 4.02156, 101.30816 4...",Kuala Penyu,10.37,2025-06-16,Ribut petir di beberapa tempat,24,33,Kuala Penyu,5.5717,115.5971,POINT (115.5971 5.5717)


In [39]:
concessions_forecast_comp['date'] = pd.to_datetime(concessions_forecast_comp['date'])

def categorize_weather(weather):
    weather = weather.lower()
    if 'hujan' in weather and 'tiada' not in weather:
        return 'Hujan'
    elif 'ribut petir' in weather:
        return 'Ribut Petir'
    elif 'berangin' in weather:
        return 'Berangin'
    elif 'tiada hujan' in weather:
        return 'Tiada Hujan'
    return weather  # fallback

concessions_forecast_comp['summary_forecast'] = concessions_forecast_comp['summary_forecast'].apply(categorize_weather)
summary_df = (
    concessions_forecast_comp.groupby(['date', 'summary_forecast'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

In [40]:
summary_df

summary_forecast,date,Berangin,Hujan,Ribut Petir,Tiada Hujan
0,2025-06-14,0,226,452,114
1,2025-06-15,0,135,620,37
2,2025-06-16,0,0,775,17
3,2025-06-17,0,61,656,75
4,2025-06-18,0,123,622,47
5,2025-06-19,0,176,460,156
6,2025-06-20,5,103,448,236


In [ ]:
df = pd.read_csv('TradeData_6_15_2025_13_30_32.csv', encoding='latin1')

In [39]:
df1 = pd.read_csv('TradeData_6_15_2025_13_30_32.csv', encoding='latin1')
df2 = pd.read_csv('TradeData_6_15_2025_15_21_46.csv', encoding='latin1')
df3 = pd.read_csv('TradeData_6_15_2025_15_22_25.csv', encoding='latin1')

In [42]:
df3.columns

Index(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period',
       'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code',
       'partner2ISO', 'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate'],
      dtype='object')

In [43]:
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

In [37]:
dff = df[['reporterISO', 'partnerISO', 'reporterDesc', 'refMonth', 'cmdCode', 'fobvalue']]

In [38]:
dff

,reporterISO,partnerISO,reporterDesc,refMonth,cmdCode,fobvalue
C,Malaysia,World,M,2023,Palm oil and its fractions; whether or not ref...,8.376651e+08
C,Malaysia,China,M,2023,Palm oil and its fractions; whether or not ref...,2.377219e+04
C,Malaysia,Ecuador,M,2023,Palm oil and its fractions; whether or not ref...,1.409224e+06
C,Malaysia,Germany,M,2023,Palm oil and its fractions; whether or not ref...,4.566481e+03
C,Malaysia,Indonesia,M,2023,Palm oil and its fractions; whether or not ref...,7.824195e+08
...,...,...,...,...,...,...
C,Malaysia,Egypt,X,2023,Vegetable oils; palm kernel or babassu oil and...,8.168563e+06
C,Malaysia,United Kingdom,X,2023,Vegetable oils; palm kernel or babassu oil and...,3.277451e+05
C,Malaysia,United Rep. of Tanzania,X,2023,Vegetable oils; palm kernel or babassu oil and...,2.170208e+04
C,Malaysia,USA,X,2023,Vegetable oils; palm kernel or babassu oil and...,3.318611e+07


In [ ]:
pip install pyvis

In [33]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML

# Assuming df is already loaded in the Notebook
# Example: df = pd.read_csv('trade_data.csv', encoding='latin1')

def create_palm_oil_graph_notebook(df):
    # Initialize directed graph
    G = nx.DiGraph()

    # Print DataFrame info for debugging
    print("DataFrame head:\n", df[['reporterDesc', 'partnerISO', 'refMonth', 'fobvalue']].head())
    print("Unique refMonth values:", df['refMonth'].unique())
    print("Unique partnerISO values:", df['partnerISO'].unique())

    # Filter out 'World' for bilateral trade
    df_bilateral = df[df['partnerISO'] != 'World'].copy()
    print(f"Number of bilateral rows: {len(df_bilateral)}")

    # Get unique countries for nodes
    countries = set(df_bilateral['reporterDesc']).union(set(df_bilateral['partnerISO']))
    print(f"Number of unique countries (nodes): {len(countries)}")

    # Add nodes
    for country in countries:
        G.add_node(country)
    print(f"Nodes added to graph: {len(G.nodes())}")

    # Add edges based on trade flow
    for _, row in df_bilateral.iterrows():
        reporter = row['reporterISO']
        partner = row['partnerISO']
        trade_flow = row['reporterDesc']
        weight = row['fobvalue']

        # Skip invalid weights but log them
        if pd.isna(weight) or weight <= 0:
            print(f"Skipping invalid weight: {weight} for {partner} -> {reporter} ({trade_flow})")
            continue

        if trade_flow == 'M':  # Import: partner -> reporter
            G.add_edge(partner, reporter, weight=weight)
            print(f"Added edge: {partner} -> {reporter} (Import, ${weight:,.2f})")
        elif trade_flow == 'X':  # Export: reporter -> partner
            G.add_edge(reporter, partner, weight=weight)
            print(f"Added edge: {reporter} -> {partner} (Export, ${weight:,.2f})")
        else:
            print(f"Unknown trade flow '{trade_flow}' for {partner} -> {reporter}")

    print(f"Edges added to graph: {len(G.edges())}")

    # Compute node degrees for sizing
    degrees = dict(G.degree())
    print("Node degrees:", {node: deg for node, deg in degrees.items()})

    # Initialize PyVis network
    net = Network(height="800px", width="100%", notebook=True, directed=True)

    # Add nodes to PyVis with size based on degree
    for node in G.nodes():
        # Scale degree to smaller size: 5-25 (e.g., degree 1 -> size 7, degree 10 -> size 25)
        node_size = 5 + (degrees[node])  # Reduced multiplier
        net.add_node(node, label=node, title=f"{node} (Degree: {degrees[node]})", size=node_size)

    # Add edges to PyVis
    for u, v, data in G.edges(data=True):
        # Scale edge width by fobvalue (1-10)
        edge_width = min(max(1, data['weight'] / 1e6), 10)  # Divide by 1M USD
        net.add_edge(u, v, value=edge_width, title=f"${data['weight']:,.2f}")
        print(f"Added PyVis edge: {u} -> {v} (width={edge_width})")

    print(f"PyVis nodes: {len(net.nodes)}, edges: {len(net.edges)}")

    # PyVis options for clear display
    net.set_options("""
    {
        "nodes": {
            "shape": "dot",
            "scaling": { "min": 10, "max": 50 },
            "font": { "size": 12 }
        },
        "edges": {
            "arrows": { "to": { "enabled": true } },
            "scaling": { "min": 1, "max": 10 }
        },
        "physics": {
            "enabled": true,
            "barnesHut": { "gravitationalConstant": -8000 }
        },
        "interaction": { "hover": true, "zoomView": true }
    }
    """)

    # Display the graph in the Notebook
    net.show("palm_oil_network.html")
    return net

# Example usage (run this cell in your Notebook)
net = create_palm_oil_graph_notebook(dff)

DataFrame head:
   reporterDesc partnerISO  refMonth      fobvalue
C            M      World      2024  1.925479e+08
C            M    Myanmar      2024  4.621618e+04
C            M   Cambodia      2024  3.717502e+05
C            M      China      2024  2.953832e+03
C            M    Ecuador      2024  6.755236e+05
Unique refMonth values: [2024]
Unique partnerISO values: ['World' 'Myanmar' 'Cambodia' 'China' 'Ecuador' 'Germany' 'Indonesia'
 'Malaysia' 'Pakistan' 'Papua New Guinea' 'Philippines' 'India'
 'Singapore' 'Thailand' 'Australia' 'Belgium' 'Brazil' 'Canada'
 'Sri Lanka' 'Fiji' 'France' 'Kiribati' 'China, Hong Kong SAR' 'Iraq'
 'Japan' 'Jordan' 'Netherlands' 'New Zealand' 'Marshall Isds' 'Viet Nam'
 'Spain' 'Sweden' 'United Arab Emirates' 'United Kingdom' 'USA'
 'Afghanistan' 'Albania' 'Algeria' 'Angola' 'Antigua and Barbuda'
 'Argentina' 'Bahrain' 'Bangladesh' 'Armenia' 'Barbados' 'Bermuda'
 'Bosnia Herzegovina' 'Belize' 'Solomon Isds' 'Brunei Darussalam'
 'Bulgaria' 'Cameroon'